In [1]:
import sys
from os.path import dirname, abspath
#_PARENT_DIR = dirname(dirname(dirname(abspath(__file__))))
#sys.path.append(_PARENT_DIR)
#from utils._utils import parse_celonis_url, get_user_data, get_logger              ### <--
from pycelonis import get_celonis
import argparse
#logger = get_logger()
def unify_tables(url, api_token, datapool, tables, tableSchema, function = 'null'):
    
    #loop through defined tables
    #for table_name in tables:
       
    c = get_celonis(url, api_token)
    #get_user_data(c, 'Table Unifier')
    #url_options= parse_celonis_url(url)                                            ### <--
    #pool = c.pools.find(url_options['id'])                                         ### <--
    
    pool = c.pools.find(datapool)                                                   ### <--

    minReq = []
    maxReq = []
    sameTables = []
    schemalist = tableSchema                                                        # added to extract required table schemas (in python list format)
    columns = ''
    select = ''
    
    # create or find global data job 
    try:
        global_dj = pool.data_jobs.find('Unify Tables')
    except:
        global_dj = pool.create_data_job(name = 'Unify Tables')
    
    #loop through defined tables
    for table_name in tables:
    
        # find tables to unify
        x = 0
        for schema in global_dj.tables:
            for table in schema:
                x = x + 1
                if table['schemaName'] not in schemalist:
                    continue
                if table['name'] == table_name:
                    sameTables.append(table)
        
        if len(sameTables) == 0:
            print('TABLE', table_name, 'CAN NOT BE UNIFIED! Not existing in source systems!')
            global_dj.create_transformation(name = "Unify " + table_name, statement = '-- no existing tables to unify in defined source systems')
            continue

        else:
            # create minimal column requirement
            for col in sameTables[0]['columns']:
                try:
                    if col in sameTables[1]['columns']:                                              # ERROR CHECKING !
                        minReq.append(col)
                except:
                    continue

            """
            #only required if function = 'min'

            if len(sameTables) > 2:
                for table in sameTables[2:]:
                    for col in minReq:
                        if col not in table['columns']:
                            minReq.pop(col)
                # create transformation statement for minimal table requirement

            if function == 'min':
                for col in minReq:
                    if(minReq.index(col) == 0):
                        columns += '{}\n'.format(col['name'])
                    else:
                        columns += ',{}\n'.format(col['name'])
                for table in sameTables:
                    select += 'SELECT \n{}FROM "{}"."{}"'.format(columns, table['schemaName'], table_name)
                    if table != sameTables[len(sameTables)-1]:
                        select += '\nUNION ALL\n'
            """            

            # create transformation for statement for extension with null
            if function == 'null':
                #columns = ''
                #select = ''
                
                for table in sameTables:
                    for col in table['columns']:
                        if col not in maxReq:
                            maxReq.append(col)
                for table in sameTables:
                    for col in maxReq:
                        if(maxReq.index(col) == 0):
                            columns += "'" + table['schemaName'] + "' " + 'AS "SCHEMA"\n' + ',{}\n'.format('"' + col['name'] + '"' if col in table['columns'] else 'NULL AS "' + col['name'] + '"')
                        else:
                            columns += ',{}\n'.format('"' + col['name'] + '"' if col in table['columns'] else 'NULL AS "' + col['name'] + '"')
                    select += 'SELECT \n{}FROM "{}"."{}"'.format(columns, table['schemaName'], table_name)
                    columns = ''
                    if table != sameTables[len(sameTables)-1]:
                        select += '\nUNION ALL\n'
            # create transformation
            #statement = 'DROP VIEW IF EXISTS "{}_UNIFIED";\n\nCreate VIEW "{}_UNIFIED" AS(\n{}\n);'.format(table_name, table_name, select)
            statement = 'DROP VIEW IF EXISTS "{}";\n\nCreate VIEW "{}" AS(\n{}\n);'.format(table_name, table_name, select)                      ### <-- Unified
            global_dj.create_transformation(name = "Unify " + table_name, statement = statement)
            print('The union statement has been saved in the global data job "Unify Tables" for Table', table_name)
            select = ''
            sameTables = []
            minReq = []
            maxReq = []
            
            # reminder: datatypes

In [2]:
# URL to the Cloud Data Pool where the tables to unify are hosted.
url = 'https://endresshauser.eu-3.celonis.cloud/'

# Specify a valid API token for your Cloud Team.
#Click on Team Settings -> Edit -> create or copy already inserted API key

api_token = 'N2JmNDI0NjMtNWQ3ZS00OGUzLTk3OTctYmJjMGM4OGE4Yjk2OnVtL3I0UldzVllJeTIwZFFIZ043R2FrTVUxRDEySFE5YWRHWnJtU2p6SkJL'


#To get the data pool id, you have to run the following code. To get the data
#the ML User here called Global needs permission to request these informations
#celonis = get_celonis()
#celonis.pools by that you get the ID for the data pool 

# Specify a datapool -- PP Data Pool ID
datapool = '590652c7-87a4-46e6-a9e9-60c90cb7c4e3'

# Specify the table names for which the tables should be merged. 

#CEPECT is empty in all systems
tables = [
'AFKO','AFPO','AFRU',
'AUFK','AUFM','CDHDR','CDPOS','CRHD','CRTX','DD02T','DD03M','JCDS','JSTO','LTAK','LTAP','MAKT','MARA','MARC','MBEW','T001','T001K','T001W','T003O','T003P','T003T','T023T','T156T','T399X','TCO43','TCORU','TCURC','TCURF','TCURR','TCURX','TCX00','TGSBT','TJ02T','TJ30T','USR02', 
]


# Specify 'min' if the unified table should be the minimal column requirement and 'null' if the not existing columns should be filled with null values.
function = 'null'

# Specify the required table schemas in a python list
#To acess the Schema Name you have to get the connection informations
#pool.data_connections
tableSchema = [
'590652c7-87a4-46e6-a9e9-60c90cb7c4e3_fff504b3-61e6-431a-9589-a0c8a1ccd3c6', # SAP ECC - PIN-020
'590652c7-87a4-46e6-a9e9-60c90cb7c4e3_bd6565ee-257a-4349-b3a2-ad0dfa097db7', # SAP ECC - PIN-024
'590652c7-87a4-46e6-a9e9-60c90cb7c4e3_9ea3d196-29ab-45e3-b825-4f6b80c543d2', # SAP ECC - PP1-011
'590652c7-87a4-46e6-a9e9-60c90cb7c4e3_f9111368-fa6e-436c-b891-ad340f7d20d8', # SAP ECC - PP2-004
'590652c7-87a4-46e6-a9e9-60c90cb7c4e3_897d99cb-b81d-4da1-b3da-d6c986f854bf' # SAP ECC - PP2-005

]


unify_tables(url = url, api_token = api_token, datapool = datapool, tables = tables, tableSchema = tableSchema, function = function)

2020-09-28 14:05:26 - pycelonis: Login successful! Hello josef.rieger@cbs-consulting.de
2020-09-28 14:05:27 - pycelonis: Best matches: [(0.44, 'Transformation PP'), (0.41, 'PP2 005 - SAP ECC PP - Full Extraction'), (0.41, 'PP2 004 - SAP ECC PP - Full Extraction')]
The union statement has been saved in the global data job "Unify Tables" for Table AFKO
The union statement has been saved in the global data job "Unify Tables" for Table AFPO
The union statement has been saved in the global data job "Unify Tables" for Table AFRU
The union statement has been saved in the global data job "Unify Tables" for Table AUFK
The union statement has been saved in the global data job "Unify Tables" for Table AUFM
The union statement has been saved in the global data job "Unify Tables" for Table CDHDR
The union statement has been saved in the global data job "Unify Tables" for Table CDPOS
TABLE CEPCT CAN NOT BE UNIFIED! Not existing in source systems!
The union statement has been saved in the global data